Problematica 
Análisis de tendencias musicales a lo largo de los años.
Queremos analizar cómo ha evolucionado la popularidad de la música en diferentes géneros y cómo algunas características como la energía, bailabilidad, acústica y loudness han cambiado a lo largo de los años. A través de este análisis, se busca entender cómo los gustos musicales han cambiado y qué características de las canciones se correlacionan más con la popularidad a lo largo del tiempo.


In [20]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import json

COdigo principla: ElasticSearchProvider.
Este código sirve para conectarse a un servidor llamado Elasticsearch que está en tu computadora. Su función principal es leer un archivo de texto donde cada línea tiene datos en formato JSON (por ejemplo, información de canciones) y luego guardar todos esos datos en una especie de base de datos llamada "spotify-insights". Lo hace todo de forma rápida y automática. Además, cuando termina de cargar los datos, cierra la conexión con el servidor para no dejarla abierta. Es útil si tienes muchos datos y quieres guardarlos en un lugar donde luego puedas buscarlos fácilmente.

In [ ]:
class ElasticSearchProvider:

    def __init__(self):
        self.host = "http://localhost:9200"
        #self.user = str(user)
        #self.password = str(password)
        self.index = "spotify-insights"
        self.index_type ="_doc"
        self.connection = Elasticsearch(self.host)

    def __enter__(self):
        try:
            self.connection=Elasticsearch(self.host)
            return self
        except Exception as e:
            return{
                "StatusCode":500,
                "body":json.dumps({
                    "Message":str(e)
                })
            }

    def insertDataJson(self, file_path):
            try:
                with open(file_path, "r", encoding="utf-8") as file:
                    documents = [
                        {"_index": self.index, "_source": json.loads(line.strip())}
                        for line in file
                    ]
                if documents:
                    helpers.bulk(self.connection, documents)
                    print(f"{len(documents)} documentos insertados en {self.index}")
            except Exception as e:
                print(f"Error al insertar datos: {str(e)}")

    def __exit__(self, exception_type, exception_value, traceback):
        self.connection.transport.close()


Lista de consultas


1.- Top 10 canciones más populares por género:

In [ ]:
def showMostPopularbyGenre(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 0,
                    "aggs": {
                        "top_genres": {
                        "terms": {
                            "field": "genre",
                            "size": 10
                        },
                        "aggs": {
                            "top_tracks": {
                            "top_hits": {
                                "size": 10,
                                "_source": ["track_name", "artist_name", "popularity"]
                                }
                            }
                        }
                    }
                }
            }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

Error: BadRequestError(400, 'media_type_header_exception', 'Invalid media-type value on headers [Accept, Content-Type]')
No se obtuvo respuesta de Elasticsearch.


2.- Top 5 canciones más populares de un artista específico.

In [ ]:
def showMostPopularbyArtist(self,name):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 5,
                    "query": {
                        "match": {
                        "artist_name": name 
                        }
                    },
                    "aggs": {
                        "top_tracks": {
                        "terms": {
                            "field": "track_name.keyword",  
                        },
                        "aggs": {
                            "popular_tracks": {
                            "top_hits": {
                                "size": 5,
                                "_source": ["track_name", "popularity"]
                            }
                            }
                        }
                        }
                    }
                }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

3.-Promedio de la popularidad por año.

In [ ]:
def showMostPopularGenrebyYear(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 0,
                    "aggs": {
                        "avg_popularity_by_genre": {
                        "terms": {
                            "field": "genre",
                            "size": 20
                        },
                        "aggs": {
                            "avg_popularity": {
                            "avg": {
                                "field": "popularity"
                            }
                            }
                        }
                        }
                    }
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

4.-Top 5 géneros más populares en un año específico.

In [ ]:
def showMostPopularGenrebySpecificYear(self,year):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 0,
                    "query": {
                        "term": {
                        "year": year
                        }
                    },
                    "aggs": {
                        "top_genres": {
                        "terms": {
                            "field": "genre",
                            "size": 5
                        },
                        "aggs": {
                            "avg_popularity": {
                            "avg": {
                                "field": "popularity"
                            }
                            }
                        }
                        }
                    }
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

5.-Top 10 canciones con la mayor energía.

In [ ]:
def showSongsWhittheMostEnegy(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 0,
                    "aggs": {
                        "top_energy_songs": {
                        "top_hits": {
                            "size": 10,
                            "_source": ["track_name", "artist_name", "energy"],
                            "sort": [
                            {
                                "energy": {
                                "order": "desc"
                                }
                            }
                            ]
                        }
                        }
                    }
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

6-. Top 10 canciones más "acústicas".

In [ ]:
def showSongsWhitMostAcustic(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 0,
                    "aggs": {
                        "top_acoustic_songs": {
                        "top_hits": {
                            "size": 10,
                            "_source": ["track_name", "artist_name", "acousticness"],
                            "sort": [
                            {
                                "acousticness": {
                                "order": "desc"
                                }
                            }
                            ]
                        }
                        }
                    }
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

7.- Promedio de "valence" por año.

In [ ]:
def showPromValencebyYear(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 0,
                    "aggs": {
                        "avg_valence_by_year": {
                        "terms": {
                            "field": "year",
                            "size": 10000  
                        },
                        "aggs": {
                            "avg_valence": {
                            "avg": {
                                "field": "valence"
                            }
                            }
                        }
                        }
                    }
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

8.- Distribución de las canciones por "mode" (mayoría mayor o menor).

In [ ]:
def showModebySongs(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 0,
                    "aggs": {
                        "mode_distribution": {
                        "terms": {
                            "field": "mode",
                            "size": 10
                        },
                        "aggs": {
                            "count_by_mode": {
                            "value_count": {
                                "field": "mode"
                            }
                            }
                        }
                        }
                    }
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

9.- Promedio de duración de canciones por género.

In [ ]:
def showDurationBySongs(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 0,
                    "aggs": {
                        "genres": {
                        "terms": {
                            "field": "genre",  
                            "size": 20
                        },
                        "aggs": {
                            "avg_duration": {
                            "avg": {
                                "field": "duration_ms" 
                            }
                            }
                        }
                        }
                    }
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

10.- Top 10 canciones más "loud".

In [ ]:
def showSongsWhitMostLoud(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 10,
                    "sort": [
                        {
                        "loudness": {
                            "order": "desc"  
                        }
                        }
                    ],
                    "_source": ["track_name", "artist_name", "loudness"]  
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

11.-Top 10 canciones con mayor "speechiness".

In [ ]:
def showSongsWhitMostSpeech(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 10,
                    "sort": [
                        {
                        "speechiness": {
                            "order": "desc"  
                        }
                        }
                    ],
                    "_source": ["track_name", "artist_name", "speechiness"] 
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

12.-Distribución de los valores de "time_signature".

In [ ]:
def showTimeSignature(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 0,
                    "aggs": {
                        "time_signature_distribution": {
                        "terms": {
                            "field": "time_signature",
                            "size": 5  
                        }
                        }
                    }
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

13.-Top 10 canciones con mayor "instrumentalness".

In [ ]:
def showInstrumental(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 10,
                    "sort": [
                        {
                        "instrumentalness": {
                            "order": "desc"
                        }
                        }
                    ],
                    "_source": ["track_name", "artist_name", "instrumentalness"]
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

14.- Las 10 canciones mas populares 

In [ ]:
def showMorePopularitySongs(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 10,
                    "sort": [
                        {
                        "popularity": {
                            "order": "desc"
                        }
                        }
                    ],
                    "_source": ["track_name", "artist_name", "popularity"]
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")

15.- las 10 canciones mas largas 

In [ ]:
def showMoreLongSongs(self):
        try:
            response=self.connection.search(
                index=self.index,
                body={
                    "size": 10,
                    "sort": [
                        {
                        "duration_ms": {
                            "order": "desc"
                        }
                        }
                    ],
                    "_source": ["track_name", "artist_name", "duration_ms"]
                    }
            )
            return response
            
        except Exception as e:
            print(f"Error: {str(e)}")